In [1]:
import pandas as pd

In [2]:
#Store Filepaths
arrestee_raw = "Resources/Arrest Data/NIBRS_ARRESTEE.csv"
incidents_raw = "Resources/Arrest Data/NIBRS_incident.csv"
agencies_raw = "Resources/Arrest Data/agencies.csv"
offence_types_raw = "Resources/Arrest Data/NIBRS_OFFENSE_TYPE.csv"
county_codes_raw = "Resources/Sex Offender Address Data/Counties.txt"

In [3]:
#read into DataFrames
arrestee_df = pd.read_csv(arrestee_raw)
incidents_df = pd.read_csv(incidents_raw)
agencies_df = pd.read_csv(agencies_raw)
offence_cats_df = pd.read_csv(offence_types_raw)
county_codes_df = pd.read_csv(county_codes_raw, sep='\t')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
#Gather the arrestees of the types related to the sex offenders database
relevant_offences_ids = [3,4,8,15,30,33,43,55,56,61,36]
relevant_offences_df = offence_cats_df.loc[offence_cats_df['OFFENSE_TYPE_ID'].isin(relevant_offences_ids)]

In [5]:
#Gather the arrestees with the found offense codes
relevant_arrests_df = arrestee_df.loc[arrestee_df['OFFENSE_TYPE_ID'].isin(relevant_offences_ids)]

In [6]:
#Get the incident IDs from the relevant arrests and create a unique list
incidents_list = relevant_arrests_df['INCIDENT_ID'].unique()
incidents_list

array([135139986, 135029010, 131873074, ..., 133092512, 133123303,
       133123356], dtype=int64)

In [7]:
#Get the incident data from the gathered incident list
relevant_incidents_df = incidents_df.loc[incidents_df['INCIDENT_ID'].isin(incidents_list)]
relevant_incidents_df.head()

,DATA_YEAR,AGENCY_ID,INCIDENT_ID,NIBRS_MONTH_ID,CARGO_THEFT_FLAG,SUBMISSION_DATE,INCIDENT_DATE,REPORT_DATE_FLAG,INCIDENT_HOUR,CLEARED_EXCEPT_ID,CLEARED_EXCEPT_DATE,INCIDENT_STATUS,DATA_HOME,ORIG_FORMAT,DID
369,2020,18918,124019051,8503244,NaN,13-MAY-20,21-JAN-20,NaN,16.0,6,NaN,0,C,F,73190451
627,2020,18988,120552482,10741384,NaN,14-FEB-20,30-JAN-20,NaN,NaN,6,NaN,0,C,F,65724864
1102,2020,19054,135031838,8487604,NaN,26-FEB-21,14-MAR-20,NaN,1.0,6,NaN,0,C,F,95992523
1115,2020,19054,135027990,8482325,NaN,26-FEB-21,29-FEB-20,NaN,17.0,6,NaN,0,C,F,95996940
1126,2020,19054,135028404,8509727,NaN,26-FEB-21,31-JAN-20,NaN,0.0,6,NaN,0,C,F,95997663


In [8]:
#get the count of incidents by offence type and agency ID
#get a list of agencies by incident
cleaned_incidents = relevant_incidents_df.filter(['AGENCY_ID','INCIDENT_ID'])

#map the incidents to arrestee data for offence code
cleaned_incidents = cleaned_incidents.merge(relevant_arrests_df,how='inner',on='INCIDENT_ID')
total_counts_df = cleaned_incidents.filter(['AGENCY_ID','INCIDENT_ID','OFFENSE_TYPE_ID'])
total_counts_df = total_counts_df.groupby(['AGENCY_ID','OFFENSE_TYPE_ID']).count()
total_counts_df = total_counts_df.reset_index()
total_counts_df

,AGENCY_ID,OFFENSE_TYPE_ID,INCIDENT_ID
0,18757,3,1
1,18757,36,2
2,18757,56,2
3,18761,36,9
4,18765,8,1
...,...,...,...
607,27352,56,1
608,27733,36,1
609,28679,36,1
610,28934,8,1


In [9]:
#Map County Name from agencies by Agency_ID
total_named_df = total_counts_df.merge(agencies_df, on='AGENCY_ID')
total_named_df = total_named_df.filter(['AGENCY_ID','OFFENSE_TYPE_ID','UCR_AGENCY_NAME','COUNTY_NAME','INCIDENT_ID'])
total_named_df = total_named_df.rename(columns={'INCIDENT_ID':'INCIDENT_COUNT'})
total_named_df.head()

,AGENCY_ID,OFFENSE_TYPE_ID,UCR_AGENCY_NAME,COUNTY_NAME,INCIDENT_COUNT
0,18757,3,ANDREWS,ANDREWS,1
1,18757,36,ANDREWS,ANDREWS,2
2,18757,56,ANDREWS,ANDREWS,2
3,18761,36,LUFKIN,ANGELINA,9
4,18765,8,ARANSAS PASS,NUECES; ARANSAS; SAN PATRICIO,1


In [10]:
#Map Offence Name from Relevant Offence list
total_named_df = total_named_df.merge(relevant_offences_df,on = 'OFFENSE_TYPE_ID')
total_df = total_named_df.filter(['AGENCY_ID','OFFENSE_TYPE_ID','UCR_AGENCY_NAME','COUNTY_NAME','OFFENSE_NAME','INCIDENT_COUNT'])
total_df

,AGENCY_ID,OFFENSE_TYPE_ID,UCR_AGENCY_NAME,COUNTY_NAME,OFFENSE_NAME,INCIDENT_COUNT
0,18757,3,ANDREWS,ANDREWS,Statutory Rape,1
1,18766,3,ROCKPORT,ARANSAS,Statutory Rape,2
2,18796,3,KILLEEN,BELL,Statutory Rape,2
3,18798,3,TEMPLE,BELL,Statutory Rape,1
4,18873,3,COLLEGE STATION,BRAZOS,Statutory Rape,1
...,...,...,...,...,...,...
607,19756,55,AUSTIN,WILLIAMSON; TRAVIS,Incest,1
608,19825,55,VERNON,WILBARGER,Incest,1
609,19870,55,HOUSTON,MONTGOMERY; FORT BEND; HARRIS,Incest,1
610,19498,33,WACO,MCLENNAN,Peeping Tom,2


In [11]:
#convert county names to lists
new_counties_list = []
for county in total_df['COUNTY_NAME']:
    county_list = county.split(';')
    new_counties_list.append(county_list)
total_df['COUNTY_NAME'] = new_counties_list
total_df

,AGENCY_ID,OFFENSE_TYPE_ID,UCR_AGENCY_NAME,COUNTY_NAME,OFFENSE_NAME,INCIDENT_COUNT
0,18757,3,ANDREWS,[ANDREWS],Statutory Rape,1
1,18766,3,ROCKPORT,[ARANSAS],Statutory Rape,2
2,18796,3,KILLEEN,[BELL],Statutory Rape,2
3,18798,3,TEMPLE,[BELL],Statutory Rape,1
4,18873,3,COLLEGE STATION,[BRAZOS],Statutory Rape,1
...,...,...,...,...,...,...
607,19756,55,AUSTIN,"[WILLIAMSON, TRAVIS]",Incest,1
608,19825,55,VERNON,[WILBARGER],Incest,1
609,19870,55,HOUSTON,"[MONTGOMERY, FORT BEND, HARRIS]",Incest,1
610,19498,33,WACO,[MCLENNAN],Peeping Tom,2


In [12]:
#Map County Codes to total_df
county_codes_df = county_codes_df.rename(columns = {'000':'COUNTY_CODE','UNKNOWN':'COUNTY_NAME'})

county_codes_list = []
for county in total_df['COUNTY_NAME']:
    codes_list = []
    for county_name in county:
        county_code = county_codes_df.loc[county_codes_df['COUNTY_NAME'] == county_name.strip()]['COUNTY_CODE'].item()
        codes_list.append(county_code)
    county_codes_list.append(codes_list)

total_df['COUNTY_CODE'] = county_codes_list
total_df

,AGENCY_ID,OFFENSE_TYPE_ID,UCR_AGENCY_NAME,COUNTY_NAME,OFFENSE_NAME,INCIDENT_COUNT,COUNTY_CODE
0,18757,3,ANDREWS,[ANDREWS],Statutory Rape,1,[2]
1,18766,3,ROCKPORT,[ARANSAS],Statutory Rape,2,[4]
2,18796,3,KILLEEN,[BELL],Statutory Rape,2,[14]
3,18798,3,TEMPLE,[BELL],Statutory Rape,1,[14]
4,18873,3,COLLEGE STATION,[BRAZOS],Statutory Rape,1,[21]
...,...,...,...,...,...,...,...
607,19756,55,AUSTIN,"[WILLIAMSON, TRAVIS]",Incest,1,"[246, 227]"
608,19825,55,VERNON,[WILBARGER],Incest,1,[244]
609,19870,55,HOUSTON,"[MONTGOMERY, FORT BEND, HARRIS]",Incest,1,"[170, 79, 101]"
610,19498,33,WACO,[MCLENNAN],Peeping Tom,2,[155]


In [14]:
total_df.to_csv('arrest_totals.csv',index=False)